In [423]:
import pandas as pd
import os
import re
import string

In [424]:
cwd = os.getcwd()
print(cwd)

C:\Users\jfortney\Anaconda3\share\jupyter\lab\Fantasy


In [425]:
basedf = pd.read_csv("nfl_pass_rush_receive_raw_data.csv")

Generate new Player_IDs and replace the old Player_IDs with them

In [426]:
# Select player-specific columns from flat file
players = basedf[["player_id", "pos", "player", "team"]]
# remove any duplicates
players = players.drop_duplicates()
# Rename columns
players.rename({'player_id': 'Player_ID', 'pos': 'Position', 'player': 'Player_Full_Name', 'team': 'Team',}, axis=1, inplace=True) 
players = (players[["Player_ID", "Position", "Player_Full_Name", "Team"]])

#Parse full name out to First Name and Last Name
out = players['Player_Full_Name'].str.extract(r"([-a-zA-Z.']+) ([-a-zA-Z.' ]+)")
#print(out)
#Insert First Name column
players["First_Name"] = out [0]
#Insert Last Name column
players["Last_Name"] = out [1]

#Remove all periods, apostrophes, hyphens and leading and trailing spaces; specifically for ID generation
players['Player_Full_forNewID'] = players['Player_Full_Name'].str.replace(r"[\'-.]", '', regex=True)
players['Player_Full_forNewID'] = players['Player_Full_forNewID'].str.strip()

#Parse full name out to First Name and Last Name (without punctuation) to recreate Player_ID
out2 = players['Player_Full_forNewID'].str.extract(r"([-a-zA-Z.']+) ([-a-zA-Z.' ]+)")
#print(out)
#Insert First Name column; remove all whitespace;
players["First_forNewID"] = out2 [0].str.strip().str.replace(r"[ ]",'', regex=True)
#Insert Last Name column; remove all whitespace;
players["Last_forNewID"] = out2 [1].str.strip().str.replace(r"[ ]",'',regex=True)

#hasnonalpha = players[players['Player_Full_Name'].str.contains(r"[.-/']", regex=True,na=False)]
lastcharoffname = players['First_forNewID'].str.strip().str[-1]
lastcharoflname = players['Last_forNewID'].str.strip().str[-1]
lastcharsofnames = players['First_forNewID'].str.strip().str[-1].map(str) + players['Last_forNewID'].str.strip().str[-1]
players["lastchars"] = lastcharsofnames.str.lower()
players["lcoffn"] = lastcharoffname.str.lower()
players["lcofln"] = lastcharoflname.str.lower()
#lastlastint = string.ascii_lowercase.index(lastcharoflname)
#print(lastlastint)
#lastfirstint = string.ascii_lowercase.index(lastcharoffname)
#print(lastfirstint)
#create ASCII integers to represent last letters of first and last names
players["llint"] = players["lcofln"].apply(ord)
players["lfint"] = players["lcoffn"].apply(ord)
#subtract the ASCII integer values from each other to get a two-digit int
players["rand2digits"] = abs(players["llint"] - players["lfint"])
#add leading zero in case of single digit int
players["rand2digits"] = players["rand2digits"].apply(lambda x: '{0:0>2}'.format(x))
#create columns for new ID (first two characters of first name; first four charachters of last name)
players["first2offirst"] = players["First_forNewID"].str.strip().str[:2]
players["first4oflast"] = players["Last_forNewID"].str.strip().str[:4]
#concatenate all the new ID fields together as a series
newplayerIDs = players["first4oflast"] + players["first2offirst"] + players["Team"] + players['Position'].str.replace(r"[\/]", '', regex=True) + players["rand2digits"]
#add new ID to dataframe
players["New_Player_ID"] = newplayerIDs
# print(newplayerIDs.duplicated().value_counts())
# print(newplayerIDs[newplayerIDs.duplicated()])
# dups = newplayerIDs[newplayerIDs.duplicated()]
# print(dups.sort_values(0)) ##HarrDeINDWR07 is a duplicate becuase he has (2) different IDs, even though he never changed teams or played multiple positions 
# print(newplayerIDs.duplicated().sum())

IDLookup = (players[["Player_ID", "New_Player_ID"]])
# IDLookup.head(50)
#Create new Player_ID column in the base df by joining basedf to players and switching them out
basedf = pd.merge(basedf, IDLookup, how='left', left_on = 'player_id', right_on = 'Player_ID')
cols = basedf.columns.tolist()
#move new joined columns to the beginning of the table
cols.insert(2, cols.pop(cols.index('Player_ID')))
cols.insert(2, cols.pop(cols.index('New_Player_ID')))
basedf = basedf.reindex(columns = cols)
#remove original 'player_id' column
basedf.drop(['player_id', 'Player_ID'], axis=1, inplace=True)
#rename New_Player_ID to Player_ID
basedf.rename({'New_Player_ID': 'Player_ID'}, axis=1, inplace=True) 
basedf.head(15)

,game_id,Player_ID,pos,player,team,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_long,pass_rating,rush_att,rush_yds,rush_td,rush_long,targets,rec,rec_yds,rec_td,rec_long,fumbles_lost,rush_scrambles,designed_rush_att,comb_pass_rush_play,comb_pass_play,comb_rush_play,Team_abbrev,Opponent_abbrev,two_point_conv,total_ret_td,offensive_fumble_recovery_td,pass_yds_bonus,rush_yds_bonus,rec_yds_bonus,Total_DKP,Off_DKP,Total_FDP,Off_FDP,Total_SDP,Off_SDP,pass_target_yds,pass_poor_throws,pass_blitzed,pass_hurried,rush_yds_before_contact,rush_yac,rush_broken_tackles,rec_air_yds,rec_yac,rec_drops,offense,off_pct,vis_team,home_team,vis_score,home_score,OT,Roof,Surface,Temperature,Humidity,Wind_Speed,Vegas_Line,Vegas_Favorite,Over_Under,game_date
0,201909050chi,RodgAaGNBQB05,QB,Aaron Rodgers,GNB,18,30,203,1,0,5,37,47,91.4,3,8,0,10,0,0,0,0,0,0,1,2,38,36,2,GNB,CHI,0,0,0,0,0,0,12.92,12.92,12.92,12.92,12.92,12.92,300,5,8,6,6,2,0,0.0,0,0,61,100,GNB,CHI,10,3,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05
1,201909050chi,JoneAaGNBRB05,RB,Aaron Jones,GNB,0,0,0,0,0,0,0,0,0.0,13,39,0,9,1,1,0,0,0,0,0,13,13,0,13,GNB,CHI,0,0,0,0,0,0,4.90,4.90,4.40,4.40,4.40,4.40,0,0,0,0,21,18,1,-1.0,1,0,37,61,GNB,CHI,10,3,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05
2,201909050chi,ValdMaGNBWR19,WR,Marquez Valdes-Scantling,GNB,0,0,0,0,0,0,0,0,0.0,1,0,0,0,6,4,52,0,47,0,0,1,1,0,1,GNB,CHI,0,0,0,0,0,0,9.20,9.20,7.20,7.20,7.20,7.20,0,0,0,0,0,0,0,81.0,2,0,41,67,GNB,CHI,10,3,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05
3,201909050chi,AdamDaGNBWR14,WR,Davante Adams,GNB,0,0,0,0,0,0,0,0,0.0,0,0,0,0,8,4,36,0,11,0,0,0,0,0,0,GNB,CHI,0,0,0,0,0,0,7.60,7.60,5.60,5.60,5.60,5.60,0,0,0,0,0,0,0,63.2,19,0,59,97,GNB,CHI,10,3,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05
4,201909050chi,GrahJiGNBTE12,TE,Jimmy Graham,GNB,0,0,0,0,0,0,0,0,0.0,0,0,0,0,5,3,30,1,16,0,0,0,0,0,0,GNB,CHI,0,0,0,0,0,0,12.00,12.00,10.50,10.50,10.50,10.50,0,0,0,0,0,0,0,58.0,17,0,33,54,GNB,CHI,10,3,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05
5,201909050chi,GrahJiCHITE12,TE,Jimmy Graham,GNB,0,0,0,0,0,0,0,0,0.0,0,0,0,0,5,3,30,1,16,0,0,0,0,0,0,GNB,CHI,0,0,0,0,0,0,12.00,12.00,10.50,10.50,10.50,10.50,0,0,0,0,0,0,0,58.0,17,0,33,54,GNB,CHI,10,3,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05
6,201909050chi,DaviTrGNBWR01,WR,Trevor Davis,GNB,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,1,28,0,28,0,0,0,0,0,0,GNB,CHI,0,0,0,0,0,0,3.80,3.80,3.30,3.30,3.30,3.30,0,0,0,0,0,0,0,18.0,10,0,17,28,GNB,CHI,10,3,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05
7,201909050chi,DaviTrLVRWR01,WR,Trevor Davis,GNB,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,1,28,0,28,0,0,0,0,0,0,GNB,CHI,0,0,0,0,0,0,3.80,3.80,3.30,3.30,3.30,3.30,0,0,0,0,0,0,0,18.0,10,0,17,28,GNB,CHI,10,3,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05
8,201909050chi,TonyRoGNBTE06,TE,Robert Tonyan,GNB,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,1,28,0,28,0,0,0,0,0,0,GNB,CHI,0,0,0,0,0,0,3.80,3.80,3.30,3.30,3.30,3.30,0,0,0,0,0,0,0,19.0,9,0,19,31,GNB,CHI,10,3,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05
9,201909050chi,WillJaGNBRB07,RB,Jamaal Williams,GNB,0,0,0,0,0,0,0,0,0.0,5,0,0,5,2,2,15,0,10,0,0,5,5,0,5,GNB,CHI,0,0,0,0,0,0,3.50,3.50,2.50,2.50,2.50,2.50,0,0,0,0,0,0,0,-6.0,21,0,24,39,GNB,CHI,10,3,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05


Create dimPlayer table

In [427]:
#Sort players by Team then by Position then by Full Name
# players.sort_values(by=['Team','Position','Player_Full_Name'], inplace=True)
players.sort_values(by=['New_Player_ID'], inplace=True)
#rename columns; switch New_Player_ID to Player_ID and Player_ID to Old_Player_ID; 
players.rename({'Player_ID': 'Old_Player_ID', 'New_Player_ID': 'Player_ID'}, axis=1, inplace=True) 
dimplayers = (players[["Player_ID", "Old_Player_ID", "Position", "Player_Full_Name", "Team", "First_Name", "Last_Name"]]).reset_index(drop = True)
#See Sample of data w/ headers
dimplayers.head(15)

,Player_ID,Old_Player_ID,Position,Player_Full_Name,Team,First_Name,Last_Name
0,AbduAmCARRB10,AbduAm00,RB,Ameer Abdullah,CAR,Ameer,Abdullah
1,AbduAmMINRB10,AbduAm00,RB,Ameer Abdullah,MIN,Ameer,Abdullah
2,AdamDaGNBWR14,AdamDa01,WR,Davante Adams,GNB,Davante,Adams
3,AdamJoNYJRB11,AdamJo03,RB,Josh Adams,NYJ,Josh,Adams
4,AdamRoCHIWR06,AdamRo01,WR,Rodney Adams,CHI,Rodney,Adams
5,AghoNeLVRWR04,AghoNe00,WR,Nelson Agholor,LVR,Nelson,Agholor
6,AghoNeNWEWR04,AghoNe00,WR,Nelson Agholor,NWE,Nelson,Agholor
7,AghoNePHIWR04,AghoNe00,WR,Nelson Agholor,PHI,Nelson,Agholor
8,AgneJaDETCB11,AgneJa00,CB,Jamal Agnew,DET,Jamal,Agnew
9,AgneJaDETRBW11,AgneJa00,RB/W,Jamal Agnew,DET,Jamal,Agnew


Create factPlays

In [428]:
# Select all the measure columns and foreign keys from the base data frame
factplayergamestats = (basedf.loc[:, ~basedf.columns.isin(['pos', 'player', 'team', 'vis_team', 'home_team', 'vis_score', 'home_score', 
                                                 'OT', 'Roof', 'Surface', 'Temperature', 'Humidity', 'Wind_Speed', 'Vegas_Line', 'Vegas_Favorite', 'Over_Under', 'game_date'])])
#move team columns to the front
factplayergamestats.rename({'Opponent_abbrev': 'Oppo_abbrev'}, axis=1, inplace=True) 
cols2 = factplayergamestats.columns.tolist()
cols2.insert(2, cols2.pop(cols2.index('Team_abbrev')))
cols2.insert(3, cols2.pop(cols2.index('Oppo_abbrev')))
factplayergamestats = factplayergamestats.reindex(columns = cols2)
factplayergamestats.head(15)

C:\Users\jfortney\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,game_id,Player_ID,Team_abbrev,Oppo_abbrev,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_long,pass_rating,rush_att,rush_yds,rush_td,rush_long,targets,rec,rec_yds,rec_td,rec_long,fumbles_lost,rush_scrambles,designed_rush_att,comb_pass_rush_play,comb_pass_play,comb_rush_play,two_point_conv,total_ret_td,offensive_fumble_recovery_td,pass_yds_bonus,rush_yds_bonus,rec_yds_bonus,Total_DKP,Off_DKP,Total_FDP,Off_FDP,Total_SDP,Off_SDP,pass_target_yds,pass_poor_throws,pass_blitzed,pass_hurried,rush_yds_before_contact,rush_yac,rush_broken_tackles,rec_air_yds,rec_yac,rec_drops,offense,off_pct
0,201909050chi,RodgAaGNBQB05,GNB,CHI,18,30,203,1,0,5,37,47,91.4,3,8,0,10,0,0,0,0,0,0,1,2,38,36,2,0,0,0,0,0,0,12.92,12.92,12.92,12.92,12.92,12.92,300,5,8,6,6,2,0,0.0,0,0,61,100
1,201909050chi,JoneAaGNBRB05,GNB,CHI,0,0,0,0,0,0,0,0,0.0,13,39,0,9,1,1,0,0,0,0,0,13,13,0,13,0,0,0,0,0,0,4.90,4.90,4.40,4.40,4.40,4.40,0,0,0,0,21,18,1,-1.0,1,0,37,61
2,201909050chi,ValdMaGNBWR19,GNB,CHI,0,0,0,0,0,0,0,0,0.0,1,0,0,0,6,4,52,0,47,0,0,1,1,0,1,0,0,0,0,0,0,9.20,9.20,7.20,7.20,7.20,7.20,0,0,0,0,0,0,0,81.0,2,0,41,67
3,201909050chi,AdamDaGNBWR14,GNB,CHI,0,0,0,0,0,0,0,0,0.0,0,0,0,0,8,4,36,0,11,0,0,0,0,0,0,0,0,0,0,0,0,7.60,7.60,5.60,5.60,5.60,5.60,0,0,0,0,0,0,0,63.2,19,0,59,97
4,201909050chi,GrahJiGNBTE12,GNB,CHI,0,0,0,0,0,0,0,0,0.0,0,0,0,0,5,3,30,1,16,0,0,0,0,0,0,0,0,0,0,0,0,12.00,12.00,10.50,10.50,10.50,10.50,0,0,0,0,0,0,0,58.0,17,0,33,54
5,201909050chi,GrahJiCHITE12,GNB,CHI,0,0,0,0,0,0,0,0,0.0,0,0,0,0,5,3,30,1,16,0,0,0,0,0,0,0,0,0,0,0,0,12.00,12.00,10.50,10.50,10.50,10.50,0,0,0,0,0,0,0,58.0,17,0,33,54
6,201909050chi,DaviTrGNBWR01,GNB,CHI,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,1,28,0,28,0,0,0,0,0,0,0,0,0,0,0,0,3.80,3.80,3.30,3.30,3.30,3.30,0,0,0,0,0,0,0,18.0,10,0,17,28
7,201909050chi,DaviTrLVRWR01,GNB,CHI,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,1,28,0,28,0,0,0,0,0,0,0,0,0,0,0,0,3.80,3.80,3.30,3.30,3.30,3.30,0,0,0,0,0,0,0,18.0,10,0,17,28
8,201909050chi,TonyRoGNBTE06,GNB,CHI,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,1,28,0,28,0,0,0,0,0,0,0,0,0,0,0,0,3.80,3.80,3.30,3.30,3.30,3.30,0,0,0,0,0,0,0,19.0,9,0,19,31
9,201909050chi,WillJaGNBRB07,GNB,CHI,0,0,0,0,0,0,0,0,0.0,5,0,0,5,2,2,15,0,10,0,0,5,5,0,5,0,0,0,0,0,0,3.50,3.50,2.50,2.50,2.50,2.50,0,0,0,0,0,0,0,-6.0,21,0,24,39


Create dimGame table

In [441]:
#select all the columns associated to games
games = basedf[["game_id", "vis_team", "home_team", "vis_score", "home_score", "OT", "Roof", "Surface", "Temperature", "Humidity", "Wind_Speed", "Vegas_Line", "Vegas_Favorite", "Over_Under", "game_date"]]
#get a distinct list of the games
games = games.drop_duplicates()
#Add Year column
#games['Game_Year'] = df.game_id.str[:4]
games.insert(1,"Game_Year", games.game_id.str[:4], True)
games.rename({'game_id': 'Game_ID', 'vis_team': 'Visiting_Team', 'home_team': 'Home_Team', 'vis_score': 'Visitor_Score', 'home_score': 'Home_Score', 'game_date': 'Game_Date'}, axis=1, inplace=True) 
dimgames = (games[["Game_ID", "Game_Year", "Game_Date", "Visiting_Team", "Home_Team", "Visitor_Score", "Home_Score", "OT", "Roof", "Surface", "Temperature", "Humidity", "Wind_Speed", 
                "Vegas_Line", "Vegas_Favorite", "Over_Under"]])

def get_season(row):
  date_ser = pd.to_datetime(row['Game_Date'])
  return date_ser.year -1 if (date_ser.month >= 1 and date_ser.month <= 2) else date_ser.year

dimgames['Season'] = dimgames.apply(lambda row: get_season(row),axis = 1)

dimgames.head(1500)

C:\Users\jfortney\AppData\Local\Temp/ipykernel_13760/562510751.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimgames['Season'] = dimgames.apply(lambda row: get_season(row),axis = 1)


,Game_ID,Game_Year,Game_Date,Visiting_Team,Home_Team,Visitor_Score,Home_Score,OT,Roof,Surface,Temperature,Humidity,Wind_Speed,Vegas_Line,Vegas_Favorite,Over_Under,Season
0,201909050chi,2019,2019-09-05,GNB,CHI,10,3,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019
36,201909080car,2019,2019-09-08,LAR,CAR,30,27,False,outdoors,grass,87,53,3,-1.5,LAR,49.5,2019
72,201909080cle,2019,2019-09-08,TEN,CLE,43,13,False,outdoors,grass,71,55,10,-5.5,CLE,44.0,2019
108,201909080crd,2019,2019-09-08,DET,ARI,27,27,True,retractable roof (closed),grass,72,45,0,-2.5,DET,45.5,2019
146,201909080dal,2019,2019-09-08,NYG,DAL,17,35,False,retractable roof (closed),fieldturf,72,45,0,-7.0,DAL,44.0,2019
182,201909080jax,2019,2019-09-08,KAN,JAX,40,26,False,outdoors,grass,91,54,6,-3.5,KAN,49.0,2019
220,201909080mia,2019,2019-09-08,BAL,MIA,59,10,False,outdoors,grass,91,56,8,-7.0,BAL,41.0,2019
269,201909080min,2019,2019-09-08,ATL,MIN,12,28,False,dome,sportturf,72,45,0,-3.5,MIN,47.0,2019
309,201909080nwe,2019,2019-09-08,PIT,NWE,3,33,False,outdoors,grass,66,62,2,-5.5,NWE,49.0,2019
345,201909080nyj,2019,2019-09-08,BUF,NYJ,17,16,False,outdoors,fieldturf,77,46,6,-2.5,NYJ,41.0,2019


In [430]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('NFL_2022_Data.xlsx', engine='xlsxwriter')

#Write the result of the dataframes to the  worksheets of the workbook
factplayergamestats.to_excel(writer, sheet_name='factPlayerGameStats', index=False)
dimgames.to_excel(writer, sheet_name='dimGames', index=False)
dimplayers.to_excel(writer, sheet_name='dimPlayers', index=False)



# Close the Pandas Excel writer and output the Excel file.
writer.save()
# Close writer and release handle
writer.close()
writer.handles = None

C:\Users\jfortney\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
